In [1]:
MSASL_TRAIN_PATH = "./MS-ASL/MSASL_train.json"
MSASL_VAL_PATH = "./MS-ASL/MSASL_val.json"
MSASL_TEST_PATH = "./MS-ASL/MSASL_test.json"

In [2]:
import json
import pandas as pd

with open(MSASL_TRAIN_PATH) as f:
    train_data = json.load(f)

with open(MSASL_VAL_PATH) as f:
    val_data = json.load(f)

with open(MSASL_TEST_PATH) as f:
    test_data = json.load(f)

train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

train_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360.0,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360.0,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360.0,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN


In [3]:
import os
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip
from tqdm import tqdm

OUTPUT_DIR = 'videos'

os.makedirs(OUTPUT_DIR, exist_ok=True)

def download_and_trim_video(sample, save_dir):
    url = sample['url']
    label = sample['label']
    signer = sample['signer_id']
    start = float(sample['start_time'])
    end = float(sample['end_time'])

    try:
        yt = YouTube(url)
        stream = yt.streams.filter(file_extension='mp4', progressive=True).first()
        if stream is None:
            print(f"No suitable stream for: {url}")
            return None

        video_path = os.path.join(save_dir, f"{label}_{signer}.mp4")
        temp_path = os.path.join(save_dir, f"temp_{label}_{signer}.mp4")
        
        stream.download(filename=temp_path)

        clip = VideoFileClip(temp_path).subclip(start, end)
        clip.write_videofile(video_path, audio=False, verbose=False, logger=None)
        clip.close()
        os.remove(temp_path)

        return video_path

    except Exception as e:
        print(f"Failed to download or process {url}: {e}")
        return None

In [6]:
import yt_dlp
import os
from moviepy.video.io.VideoFileClip import VideoFileClip

def download_with_ytdlp(url, filename):
    ydl_opts = {
        'format': 'best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return True
    except Exception as e:
        print(f"yt-dlp failed: {e}")
        return False

def download_and_trim_video(sample, save_dir):
    url = sample['url']
    label = sample['label']
    signer = sample['signer_id']
    start = float(sample['start_time'])
    end = float(sample['end_time'])

    base = f"{label}_{signer}"
    video_path = os.path.join(save_dir, f"{base}.mp4")
    temp_path = os.path.join(save_dir, f"temp_{base}.mp4")

    if download_with_ytdlp(url, temp_path):
        try:
            clip = VideoFileClip(temp_path).subclip(start, end)
            clip.write_videofile(video_path, audio=False, verbose=False, logger=None)
            clip.close()
            os.remove(temp_path)
            return video_path
        except Exception as e:
            print(f"Failed to clip video: {e}")
    return None

In [7]:
downloaded = []

for sample in tqdm(train_data[:100]): 
    path = download_and_trim_video(sample, OUTPUT_DIR)
    if path:
        downloaded.append(path)
    if len(downloaded) >= 10:
        break

  0%|                                                                                                                   | 0/100 [00:00<?, ?it/s]

  1%|█                                                                                                          | 1/100 [00:04<08:02,  4.87s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2023-02-04T09:47:19.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 244, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2023-02-04T09:47:19.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/04/2023.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'creation_time': '2023-02-04T09:47:19.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/04/2023.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2.86, 'bitrate': 378, 'start': 0.0, 'default_video_input_nu

  2%|██▏                                                                                                        | 2/100 [00:09<07:29,  4.59s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2023-01-06T10:53:02.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 174, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2023-01-06T10:53:02.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/06/2023.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'creation_time': '2023-01-06T10:53:02.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/06/2023.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 3.04, 'bitrate': 309, 'start': 0.0, 'default_video_input_nu

...

Failed to clip video: 'VideoFileClip' object has no attribute 'subclip'
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2015-02-12T22:39:55.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 297, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2015-02-12T22:39:55.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/12/2015.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 125, 'metadata': {'Metadata': '', 'creation_time': '2015-02-12T22:39:55.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/12/2015.', 'vendor_id': '[0][0][0][0]'}}], 'input_

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [06:32<00:03,  3.48s/it]

Failed to clip video: 'VideoFileClip' object has no attribute 'subclip'
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2015-02-12T22:39:55.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 297, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2015-02-12T22:39:55.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/12/2015.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 125, 'metadata': {'Metadata': '', 'creation_time': '2015-02-12T22:39:55.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/12/2015.', 'vendor_id': '[0][0][0][0]'}}], 'input_

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:35<00:00,  3.96s/it]

Failed to clip video: 'VideoFileClip' object has no attribute 'subclip'


In [15]:
from IPython.display import HTML

def display_video(path, width=400, height=300):
    return HTML(f"""
    <video width="{width}" height="{height}" controls>
      <source src="{path}" type="video/mp4">
      Your browser does not support the video tag.
    </video>
    """)

display_video("./msasl_clips/temp_2_144.mp4")

In [16]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from pathlib import Path

FRAME_COUNT = 16
FRAME_SIZE = 224  # width/height

video_dir = Path("videos")  # where your .mp4 files are

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((FRAME_SIZE, FRAME_SIZE)),
    transforms.ToTensor(),  # Converts to [0, 1] and puts channel first
])

def load_video_frames(path, num_frames=FRAME_COUNT):
    cap = cv2.VideoCapture(str(path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        indices = np.linspace(0, total_frames - 1, total_frames).astype(int)
    else:
        indices = np.linspace(0, total_frames - 1, num_frames).astype(int)

    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = transform(frame)
            frames.append(frame)

    cap.release()

    if len(frames) < num_frames:
        # Pad with last frame if too short
        while len(frames) < num_frames:
            frames.append(frames[-1])
    return torch.stack(frames)  # Shape: (T, C, H, W)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
video_path = video_dir / "0001.mp4"
frames_tensor = load_video_frames(video_path) 